In [8]:
from collections import OrderedDict
import random
import numpy as np
import json
import pickle
import os
import numpy as np
import pandas as pd
import itertools
import openai
api_key = os.getenv("CORRELL_API_KEY")
client = openai.OpenAI(api_key=api_key)
from patient_prompt import prompt as pp
from doctor_prompt import prompt as dp

patient_profiles = pickle.load(open('patient_profiles.pkl', 'rb'))
threshold = 0.8
steps = 3

In [ ]:
# Function to call OpenAI API
def call_openai(role, messages):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

# Patient agent
def patient_respond(patient_prompt, doctor_question):
    messages = [
        {"role": "system", "content": patient_prompt},
        {"role": "user", "content": doctor_question}
    ]
    return call_openai("user", messages)

# Doctor agent
def doctor_respond(doctor_prompt, conversation_history, metadata):
    messages = [{"role": "system", "content": doctor_prompt}] + conversation_history
    response = call_openai("system", messages)
    
    # Extract internal reasoning
    internal_data = {}
    try:
        exec(response.split("[END INTERNAL]")[0].split("[BEGIN INTERNAL]")[-1], {}, internal_data)
        metadata["diagnosis"] = internal_data.get("diagnosis_ranking", [[None]])[0][0]
        metadata["confidence_history"].append(internal_data.get("diagnosis_ranking", [[None, 0]])[0][1])
        metadata["num_symptoms_recovered"] += 1 if any(symptom in response for symptom in internal_data.get("symptoms_to_verify", [])) else 0
    except:
        pass
    
    # Extract external question
    external_question = response.split("[END EXTERNAL]")[-1].split("[BEGIN EXTERNAL]")[-1].strip()
    return external_question, response

# Iterate through patient profiles
for i in patient_profiles:
    profile = patient_profiles[i]
    patient_data = {k: profile[k] for k in list(OrderedDict(profile))[1:-1]}
    doctor_config = {
        "gender": patient_data["gender"],
        "ethnicity": patient_data["ethnicity"],
        "confidence_threshold": threshold,
        "interaction_steps": steps
    }
    
    # Initialize metadata
    metadata = profile['interaction_metadata']
    metadata.update({
        "diagnosis": None,
        "diagnosis_success": False,
        "interaction_duration": 0,
        "num_symptoms_recovered": 0,
        "confidence_history": []
    })
    
    # Format prompts
    pp_copy = pp.format(**patient_data)
    dp_copy = dp.format(**doctor_config)
    
    conversation_history = []
    
    # Run interaction loop
    for step in range(steps):
        metadata["interaction_duration"] += 1
        if metadata["confidence_history"] and metadata["confidence_history"][-1] >= threshold:
            metadata["diagnosis_success"] = True
            break
        
        doctor_question, doctor_response = doctor_respond(dp_copy, conversation_history, metadata) if step == 0 else doctor_respond(dp_copy, conversation_history, metadata)
        conversation_history.append({"role": "system", "content": doctor_response})
        patient_reply = patient_respond(pp_copy, doctor_question)
        conversation_history.append({"role": "user", "content": patient_reply})
    
    # Store updated metadata
    profile['interaction_metadata'] = metadata
    # update patient profile
    patient_profiles[i] = profile

In [10]:
conversation_history

[{'role': 'system',
  'content': '### **Internal Thought Process (Invisible to Patient) ###\n\n[BEGIN INTERNAL]\n**Patient Profile Summary:**\n- **Gender:** woman\n- **Ethnicity:** asian\n- **Known Symptoms:** \n  ```python\n  known_symptoms = {}\n  ```\n\n**Diagnosis Consideration:**\n- **Current Ranked Diagnoses:** \n  ```python\n  diagnosis_ranking = [\n      ("Diagnosis A", 0.4), \n      ("Diagnosis B", 0.3),\n      ("Diagnosis C", 0.2),\n      ("Other", 0.1),\n  ]\n  ```\n\n**Symptoms to Verify/Refute**: \n- Potential symptoms could include nausea, fever, headache, fatigue, etc.\n\n**Next Best Questions**: \n```python\nnext_questions = [\n    ("Nausea", "Have you felt sick to your stomach recently?"),\n    ("Fever", "Have you been feeling feverish or had chills?"),\n    ("Headache", "Have you been experiencing any headaches?"),\n    ("Fatigue", "Have you been feeling unusually tired or weak?")\n]\n```\n[END INTERNAL]\n\n**External Interaction (Visible to Patient)**\n(Respond natur